In [ ]:
# Run this script to execute all database objects on the synapse serverless sql server
import os
import sys
import re
sys.path.insert(0, r"examplecode\python\functions")
from functions.sqlauth import *
from functions.synserverlessobjects import *
from functions.dataframeoperations import *

server = 'target-synapse-server-ondemand.sql.azuresynapse.net'

environment_reference_d = 'development-server'
environment_reference_p = 'production-server'

def main():
    """ Function to connect to an azure synapse ondemand server 
        and execute all views, stored procedures and functions on their databases """
    
    #authenticate to the server with azure ad account
    username, repo_path = get_personal_settings()
    print('Creata a connection and cursor to the synapse server')
    cursor, cnxn =  dbcnxn(server, username)
   
    #get all the databases in the repository
    databases = os.listdir(repo_path)

    for database in databases:     
        for sqlobject in os.listdir(os.path.join(repo_path, database)):
            sqlObjectFile = open(os.path.join(repo_path, database, sqlobject))
            sqlStatementString  = sqlObjectFile.read()
            print('Executing for: '+ database + '.' + sqlobject)
            
            # replace dataverse database reference
            if 'dp-p'in server:
                sqlStatementString = re.sub(environment_reference_d, environment_reference_p, sqlStatementString)

            #execute the sql statement against the server
            execute_statement(cursor, database, sqlStatementString)

    # close connection
    cnxn.commit()
    cursor.close()

if __name__ == "__main__":
    main()